# 05 벡터의 유사도 Vector Similarity


### 1) 코사인 유사도 Cosine Similarity

1. 코사인 유사도 : 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터 간의 유사도

  두 벡터 방향이 동일한 경우 1

  90도 각을 이루면 0

  180도로 반대의 방향을 가지면 -1

  즉, -1 이상 1 이하 값을 가지며, 1에 가까울수록 유사도가 높다.

In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B)) # 두 문서의 코사인 유사도

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print('문서 1과 문서2의 유사도 :',cos_sim(doc1, doc2))
print('문서 1과 문서3의 유사도 :',cos_sim(doc1, doc3))
print('문서 2와 문서3의 유사도 :',cos_sim(doc2, doc3))

문서 1과 문서2의 유사도 : 0.6666666666666667
문서 1과 문서3의 유사도 : 0.6666666666666667
문서 2와 문서3의 유사도 : 1.0000000000000002


2. 유사도를 이용한 추천 시스템 구현 : 캐글 영화 데이터 셋

  https://www.kaggle.com/rounakbanik/the-movies-dataset

  movies_metadata.csv 파일 사용

In [3]:
from google.colab import files

myfile = files.upload()

Saving movies_metadata.csv to movies_metadata.csv


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [5]:
data = data.head(2000)
data.isnull().sum()

adult                       0
belongs_to_collection    1680
budget                      0
genres                      0
homepage                 1874
id                          0
imdb_id                     0
original_language           0
original_title              0
overview                   16
popularity                  0
poster_path                13
production_companies        0
production_countries        0
release_date                2
revenue                     0
runtime                     5
spoken_languages            0
status                      3
tagline                   492
title                       0
video                       0
vote_average                0
vote_count                  0
dtype: int64

In [9]:
data['overview'] = data['overview'].fillna('')

tfidf = TfidfVectorizer(stop_words='english') # TF-IDF
tfidf_matrix = tfidf.fit_transform(data['overview'])

cosine_sim = cosine_similarity(tfidf_matrix,tfidf_matrix) # 코사인 유사도

title_to_index = dict(zip(data['title'],data.index))

def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return data['title'].iloc[movie_indices]

get_recommendations('Toy Story')

1071       Rebel Without a Cause
1932                   Condorman
485                       Malice
448            For Love or Money
1032               The Sunchaser
1884              Child's Play 3
748           I Shot Andy Warhol
591              Window to Paris
314     The Shawshank Redemption
180                 Mute Witness
Name: title, dtype: object

### 2) 여러가지 유사도 기법

1. 유클리드 거리 : 두 점 사이 거리

2. 자카드 유사도 : 합집합에서 교집합의 비율


In [10]:
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

# 합집합
union = set(tokenized_doc1).union(set(tokenized_doc2))

# 교집합
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))

print('자카드 유사도 :',len(intersection)/len(union))

자카드 유사도 : 0.16666666666666666
